# Instructions

This program is designed to be used in tandem with GEE_data_generation. It will combine all of the resultant csv files, drop the unnecessary columns, and sort the rows. Using this notebook is slightly more complicated than the previous one. There are a few parameters that you can change.
* Setting `REMOVE_CSV` to True will remove all of the .csv files in the working directory (except the one this program generates)
* Setting `TITLE_CASE` to True will make sure each column header is capitalized.
* `CSV_NAME` is what the processed csv will be called.

**RUN EVERY CELL ONE AT A TIME.** If you do not, you risk missing the second user variable section.
In the second user variable section you may specify what columns to keep and which columns to use as sort keys.
* `DESIRED_COLUMNS` should be a square-bracketed list with the column names exactly as they appear in the table as a string. The order you specify in this list will be the final order in the exported csv file.
* The list `SORT_KEYS` represents the keys by which this program will sort the DataFrame. If you want to sort by either year or month, the appropriate name is 'Date' as opposed to year or month. The order you specify in this list determines the sort order.
    * Putting 'DHSCLUST' before 'Date' will result in all of the DHSCLUST=1 sorted by date before the first DHSCLUST=2 appears
    * Putting 'Date' before 'DHSCLUST' will result in a table where all of the clusters are sorted by date. If the first input month is 4-2014, the exported csv file will have DHSCLUST=1 at 4-2014 then DHSCLUST=2 at 4-2014 and so on.


## User Variables (pt. 1)

In [91]:
REMOVE_CSV = True
TITLE_CASE = True
CSV_NAME = 'final_data.csv'

# Initial Setup

In [92]:
# Dependancies
import pandas as pd
import os
from functools import reduce

# Map functions
def combine_df(master, to_be_combined):
    return pd.concat([master, to_be_combined], axis=0)  

def get_column_loc(name):
    col = master_df.columns
    return col.get_loc(name)

# modify CSV_NAME if needed
if '.csv' not in CSV_NAME: CSV_NAME += '.csv'

## Combine CSV and Print Column Names

In [93]:
# make a list of all .csv files in the working directory
os.system('ls *.csv > csv_names.txt')
with open('csv_names.txt', 'r') as fp:
    names = fp.read()
name_list = names.split('\n')
name_list.pop()


# turn the list of .csv files into a list of Panda DataFrames and combine
df_list = map(lambda csv: pd.read_csv(csv), name_list)
master_df = reduce(combine_df, df_list)

# capitalize column names if desired
if TITLE_CASE:
    new_columns = {}
    for column in master_df.columns:
        if column[0] != column.upper()[0]:
            new_columns[column] = column.title()
        else: new_columns[column] = column
    master_df.rename(columns=new_columns, inplace=True)

# print out the column names and the top 5 rows of the master DataFrame
for column in master_df.columns:
    print (column)
master_df.head()

Unnamed: 0
Mean
ADM1FIPS
ADM1NAME
CCFIPS
ALT_GPS
ADM1SALBNA
DHSREGCO
ALT_DEM
DHSID
LATNUM
DHSREGNA
DHSCC
ADM1SALBCO
DATUM
DHSYEAR
URBAN_RURA
DHSCLUST
SOURCE
ADM1DHS
ADM1FIPSNA
System:Index
LONGNUM
Month


,Unnamed: 0,Mean,ADM1FIPS,ADM1NAME,CCFIPS,ALT_GPS,ADM1SALBNA,DHSREGCO,ALT_DEM,DHSID,...,DATUM,DHSYEAR,URBAN_RURA,DHSCLUST,SOURCE,ADM1DHS,ADM1FIPSNA,System:Index,LONGNUM,Month
0,0,0.170966,ET44,addis abeba,ET,9999.0,Addis Ababa City,14.0,2130.0,ET200500000454,...,WGS84,2005.0,R,454.0,GPS,14.0,Adis Abeba,00000000000000000114,38.785554,1-2015
1,1,0.171385,ET44,addis abeba,ET,9999.0,Addis Ababa City,14.0,2363.0,ET200500000295,...,WGS84,2005.0,R,295.0,GPS,14.0,Adis Abeba,000000000000000001c2,38.876145,1-2015
2,2,0.172684,ET44,addis abeba,ET,9999.0,Addis Ababa City,14.0,2332.0,ET200500000002,...,WGS84,2005.0,U,2.0,GPS,14.0,Adis Abeba,00000000000000000001,38.793397,1-2015
3,3,0.177433,ET44,addis abeba,ET,9999.0,Addis Ababa City,14.0,2515.0,ET200500000019,...,WGS84,2005.0,U,19.0,GPS,14.0,Adis Abeba,00000000000000000012,38.750210,1-2015
4,4,0.175763,ET44,addis abeba,ET,9999.0,Addis Ababa City,14.0,2497.0,ET200500000020,...,WGS84,2005.0,U,20.0,GPS,14.0,Adis Abeba,00000000000000000013,38.804187,1-2015


## User Variables (pt. 2)

In [94]:
# specify the desired columns
DESIRED_COLUMNS = ['ADM1SALBNA', 'DHSCLUST', 'Month', 'Mean']

# keys by which to sort
SORT_KEYS = ['DHSCLUST', 'Date']

## Process DataFrame

In [95]:
# drop the irrelevant columns
for column in master_df.columns:
    if column not in DESIRED_COLUMNS:
        master_df.drop(column, inplace=True, axis=1)

# re-order the relevant ones
column_order = list(map(get_column_loc, DESIRED_COLUMNS))
current_order = master_df.columns.tolist()
final_order = [current_order[i] for i in column_order]
master_df = master_df[final_order]

# sort DataFrame
master_df['Date'] = pd.to_datetime(master_df.Month)
master_df.sort_values(by=SORT_KEYS, ignore_index=True, inplace=True)
master_df.drop('Date', inplace=True, axis=1)

# export to csv
master_df.to_csv(CSV_NAME, index=False)

## Clean up Working Directory

In [96]:
if REMOVE_CSV:
    for i in name_list:
        os.system('rm ' + i)
os.system('rm csv_names.txt')

0

# Credits

* Created by Caleb Bitting